In [1]:
import numpy as np
import pandas as pd

In [2]:
# Data pre-processing
data = pd.read_csv('data/data_elog_eindhoven.csv', sep = ';')
# Delete columns
to_delete = ['Unnamed: 0', 'index']
data.drop(to_delete, axis=1, inplace=True)

#Sorth the data
data.sort_values(['location', 'UTC_time' ], ascending=[True, True], inplace=True)
def calculate_diff(data): 
    """
    In this function the consumption difference is calculated per user.
    """
    def diff_func(df): return df.diff()
    data['delta_total'] = data.groupby('location')['total'].apply(diff_func)
    
    return data.reset_index(drop=True)

data = calculate_diff(data)

#Create new varianbles
data['dummy'] = 1
data['datetime64'] = pd.to_datetime(data['UTC_time'])
data['norm_date'] = data['datetime64'].dt.normalize()
data['year'] = data['datetime64'].dt.year
data['month'] = data['datetime64'].dt.month
data['day'] = data['datetime64'].dt.day
data['hour'] = data['datetime64'].dt.hour
data = data[data['year'] == 2017] #Only files in 2017
data.to_csv('data/data_processed.csv')

# Data Agregation

In [7]:
def data_aggregation(data, aggegation_method = 'sum'):
    """
    This funtion creates the matrices that will be use to generate the heat maps
    Params:
    data: the elog data set
    aggegation_method: how to aggregate the data ['sum', 'mean', 'median']
    Return:
    hour_consuption: Matrix with the average water consuption per time slot (hour)
    """
    # Here we create the matrices that will be shown at the heat-map
    data = data.dropna()
    
    if aggegation_method == 'median':
        hour_consuption = data.groupby(by = ['norm_date', 'hour'])['delta_total'].median()
   
    elif aggegation_method == 'sum':
        hour_consuption = data.groupby(by = ['norm_date', 'hour'])['delta_total'].sum()
        
    elif aggegation_method == 'mean':
        hour_consuption = data.groupby(by = ['norm_date', 'hour'])['delta_total'].mean() 
        
    else:
        print('The option {} does not exist, please select [sum, mean, median]'.format(aggegation_method))
        sys.exit()
        
    num_locations = data.groupby(by = ['norm_date', 'hour'] , as_index=False).apply(lambda x: x.location.nunique()) #This must be cheched, all values are 5
    # Change formats
    def format_change(df):
#         dict_months = {'1':'Jan', '2':'Feb', '3':'Mar', '4':'Apr', '5':'May', '6':'Jun', '7':'Jul', '8':'Aug', '9':'Sep', '10':'Oct', '11':'Nov', '12':'Dec'}
        df = df.unstack()
        df.index = df.index.astype(str)
        df.columns = df.columns.astype(str)  
        df = df.T
#         names = []
#         for c in df.columns:
#             names.append(dict_months[c])
#         df.columns = pd.Series(names)
        df.columns.name = 'date'
        return df
    
    hour_consuption = format_change(hour_consuption)
    num_locations = format_change(num_locations)
    
    assert hour_consuption.shape == num_locations.shape, 'different shapes'

    return hour_consuption, num_locations

hour_consuption, num_locations = data_aggregation(data, 'median')

In [9]:
# Create a file per location
def create_files_HM(data, total = False):
    unique_location = data['location'].unique()
    
    for i in unique_location:
        temp_data = data[data['location'] == i]
        hour_consuption, num_locations = data_aggregation(temp_data, 'sum')
        aggregated_day = hour_consuption.sum(axis=0)
        aggregated_day = aggregated_day.reset_index()
        aggregated_day.columns = ['norm_date', 'total_consuption']
        
        hour_consuption.to_csv('data/Data_heat_maps/hour_consuption/{}.csv'.format(str(i)),index = False)
        num_locations.to_csv('data/Data_heat_maps/num_locations/{}.csv'.format(str(i)), index = False)
        aggregated_day.to_csv('data/Data_heat_maps/aggregated_day/{}.csv'.format(str(i)), index = False)
    
    if total:
        aggregated_day_total = pd.DataFrame(data.groupby(by = ['location', 'norm_date'])['delta_total'].sum())
        aggregated_day_total.to_csv('data/Data_heat_maps/aggregated_day/aggregated_day_total.csv')
        hour_consuption, num_locations = data_aggregation(data, 'median')
        hour_consuption.to_csv('data/Data_heat_maps/hour_consuption/hour_consuption_total_median.csv',index = False)
        num_locations.to_csv('data/Data_heat_maps/num_locations/num_locations_total_median.csv',index = False)
                           
create_files_HM(data, True)

In [10]:
data_hp = pd.read_csv('data/Data_heat_maps/hour_consuption/1163208.csv')
data_aggregated = pd.read_csv('data/Data_heat_maps/aggregated_day/aggregated_day_total.csv')

In [11]:
data_hp

,2017-02-09,2017-02-10,2017-02-11,2017-02-12,2017-02-13,2017-02-14,2017-02-15,2017-02-16,2017-02-17,2017-02-18,...,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31
0,NaN,162.0000,228.6666,160.0,150.0,120.0,180.0,130.0,70.0,80.0,...,1320.0,1220.0,1220.0,1240.0,1300.0,1210.0,1930.0,1220.0,1180.0,1220.0
1,NaN,168.6667,201.3334,180.0,160.0,200.0,120.0,110.0,60.0,120.0,...,1360.0,1220.0,1230.0,1230.0,1240.0,1280.0,1940.0,1270.0,1230.0,1210.0
2,NaN,210.0000,190.6666,160.0,140.0,180.0,140.0,80.0,80.0,70.0,...,1230.0,1230.0,1210.0,1220.0,1280.0,1290.0,1930.0,1240.0,1280.0,1190.0
3,NaN,280.0000,226.6667,170.0,150.0,190.0,190.0,380.0,210.0,110.0,...,1200.0,1300.0,1300.0,1220.0,1230.0,1220.0,1950.0,1260.0,1220.0,1230.0
4,NaN,468.0000,196.0000,220.0,400.0,390.0,550.0,250.0,230.0,160.0,...,1190.0,1300.0,1240.0,1280.0,1250.0,1250.0,1980.0,1230.0,1280.0,1190.0
5,NaN,558.6666,250.0000,460.0,760.0,950.0,570.0,720.0,430.0,110.0,...,1540.0,1320.0,1340.0,1280.0,1300.0,1710.0,2380.0,1760.0,1350.0,1290.0
6,NaN,720.0000,330.0000,350.0,880.0,800.0,1260.0,1180.0,810.0,290.0,...,1820.0,1290.0,1330.0,1460.0,1650.0,1640.0,2270.0,1920.0,1600.0,1470.0
7,NaN,763.3334,420.0000,300.0,2230.0,1590.0,1410.0,1170.0,820.0,260.0,...,1980.0,1360.0,1410.0,1450.0,1610.0,2310.0,2070.0,2050.0,1390.0,1360.0
8,NaN,1084.0000,370.0000,340.0,1770.0,2540.0,1540.0,1920.0,1050.0,400.0,...,2060.0,1470.0,1390.0,1470.0,1490.0,1950.0,1890.0,3310.0,1530.0,1390.0
9,NaN,1887.3333,470.0000,410.0,1670.0,1770.0,1710.0,1400.0,1130.0,410.0,...,1870.0,1360.0,1340.0,1480.0,1480.0,1950.0,2040.0,6520.0,1440.0,1320.0


In [12]:
data_aggregated.head()

,location,norm_date,delta_total
0,1163208,2017-02-09,6446.0000
1,1163208,2017-02-10,16070.6667
2,1163208,2017-02-11,8593.3333
3,1163208,2017-02-12,9430.0000
4,1163208,2017-02-13,21080.0000
